In [ ]:
# 每个MDP，都有一套V*/Q*，有多个Pi*对应到值*
# 对于单个MDP，从不同Pi出发，可以达到不同的Pi*，但是值*唯一
# 不同Pi在MDP下都会有一套稳定的值V/Q

In [ ]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py  # 左、下、右、上

# https://github.com/boyu-ai/Hands-on-RL

# https://zhuanlan.zhihu.com/p/468235479
# https://zhuanlan.zhihu.com/p/470994332
# https://zhuanlan.zhihu.com/p/473798126

# https://blog.csdn.net/jiebaoshayebuhui/article/details/128487283
# https://zhuanlan.zhihu.com/p/54936262

In [ ]:
# 环境搭建：
# conda create -n RL_Basics
# conda activate RL_Basics

# conda install python=3.8

# https://pytorch.org/get-started/previous-versions/
# conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia



# conda install ipykernel
# conda install platformdirs
# pip3 install ipywidgets
# pip3 install --upgrade jupyter_core jupyter_client

# python -m ipykernel install --user --name RL_Basics

In [ ]:
# pip install "gymnasium[all]"
# pip install swig

In [ ]:
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

import copy
import matplotlib.pyplot as plt  # https://matplotlib.org/stable/index.html#
import numpy as np
import random

In [ ]:
def equiprobability_policy(env) :

    pi = np.ones( [env.observation_space.n, env.action_space.n] )  # 16x4

    pi = pi * (1/env.action_space.n)

    return pi

In [ ]:
def episode_sample(env, pi) :

    obs, _ = env.reset()

    episode = []
    while True :
        timestep = []

        timestep.append(obs)

        action = np.random.choice(env.action_space.n, p=pi[obs])

        next_obs, reward, terminated, truncated, _ = env.step(action)

        timestep.append(action)
        timestep.append(reward)
        timestep.append(next_obs)
        episode.append(timestep)

        obs = next_obs

        if terminated or truncated :
            break

    return episode

In [ ]:
def policy_evaluation_V(env, pi, epsilon=0.10, gamma=0.98, theta=1e-6) :

    old_V = np.zeros(env.observation_space.n)
    new_V = np.zeros(env.observation_space.n)

    old_Q = np.zeros( [env.observation_space.n, env.action_space.n] )
    new_Q = np.zeros( [env.observation_space.n, env.action_space.n] )

    cnt = 0
    while True :
        diff = 0

        for s in range(env.observation_space.n) :
            new_V[s] = 0

            for a in range(env.action_space.n) :
                new_Q[s, a] = 0

                # print(env.P[s][a])
                for p, s_next, r, done in env.P[s][a] :  # if s_next is done
                    new_Q[s, a] += p*(r+gamma*old_V[s_next]*(1-done))

                new_V[s] += pi[s, a]*new_Q[s, a]

            diff += np.abs(new_V[s] - old_V[s])
            old_V[s] = new_V[s]
        # old_V = new_V  # 直接赋值，精度不够？变量和变量的=，变化会同步

        cnt += 1

        if diff < theta :
            print(cnt)
            break

    return old_V

In [ ]:
def policy_evaluation_Q(env, pi, epsilon=0.10, gamma=0.98, theta=1e-6) :

    old_V = np.zeros(env.observation_space.n)
    new_V = np.zeros(env.observation_space.n)

    old_Q = np.zeros( [env.observation_space.n, env.action_space.n] )
    new_Q = np.zeros( [env.observation_space.n, env.action_space.n] )

    cnt = 0
    while True :
        diff = 0

        for s in range(env.observation_space.n) :
            new_V[s] = 0

            for a in range(env.action_space.n) :
                new_Q[s, a] = 0

                # print(env.P[s][a])
                for p, s_next, r, done in env.P[s][a] :  # if s_next is done
                        new_V[s_next] = 0

                        for a_next in range(env.action_space.n) :
                            new_V[s_next] += pi[s_next, a_next]*old_Q[s_next, a_next]

                        new_Q[s, a] += p*(r+gamma*new_V[s_next]*(1-done))

                diff += np.abs(new_Q[s, a] - old_Q[s, a])
                old_Q[s, a] = new_Q[s, a]

        for s in range(env.observation_space.n) :
            new_V[s] = 0
            for a in range(env.action_space.n) :
                new_V[s] += pi[s, a]*old_Q[s, a]
            old_V[s] = new_V[s]

        cnt += 1

        if diff < theta :
            print(cnt)
            print(old_V)
            break

    return old_Q

In [ ]:
def mc_evaluation_V(env, pi, n_episodes=500000, gamma=0.98) :

    new_V = np.zeros(env.observation_space.n)
    cnt_V = np.zeros(env.observation_space.n)

    for i in range(n_episodes) :

        episode = episode_sample(env, pi)
        G = 0

        for j in range( len(episode)-1, -1, -1 ) :  # range(i, j) == i ~ j-1

            (s, a, r, s_next) = episode[j]
            G = r + gamma*G

            cnt_V[s] = cnt_V[s] + 1
            new_V[s] = new_V[s] + (G - new_V[s])/cnt_V[s]

    return new_V

In [ ]:
def mc_evaluation_Q(env, pi, n_episodes=500000, gamma=0.98) :

    new_Q = np.zeros( [env.observation_space.n, env.action_space.n] )
    cnt_Q = np.zeros( [env.observation_space.n, env.action_space.n] )

    for i in range(n_episodes) :

        episode = episode_sample(env, pi)
        G = 0

        for j in range( len(episode)-1, -1, -1 ) :  # range(i, j) == i ~ j-1

            (s, a, r, s_next) = episode[j]
            G = r + gamma*G

            cnt_Q[s, a] = cnt_Q[s, a] + 1
            new_Q[s, a] = new_Q[s, a] + (G - new_Q[s, a])/cnt_Q[s, a]

    return new_Q

In [ ]:
def TD0_evaluation_V(env, pi, n_episodes=500000, gamma=0.98, alpha=0.05) :

    new_V = np.zeros(env.observation_space.n)

    for i in range(n_episodes) :
        episode = episode_sample(env, pi)

        for j in range( len(episode) ) :
            (s, a, r, s_next) = episode[j]  # expolit all the transitions
            new_V[s] += alpha*(r+gamma*new_V[s_next]-new_V[s])

    return new_V

In [ ]:
def TDlambda_evaluation_V(env, pi, n_episodes=500000, gamma=0.98, TDlambda=0.98, alpha=0.05) :

    new_V = np.zeros(env.observation_space.n)
    new_T = np.zeros(env.observation_space.n)

    for i in range(n_episodes) :
        episode = episode_sample(env, pi)

        for j in range( len(episode) ) :
            (s, a, r, s_next) = episode[j]

            # 和transitions利用的顺序有关？短时间高强度访问，增大学习权重
            new_T = gamma*TDlambda*new_T
            new_T[s] += 1

            new_V[s] += alpha*(r+gamma*new_V[s_next]-new_V[s])*new_T[s]  # 放大了学习程度

    return new_V

In [ ]:
def policy_improvement_greedy(env, Q) :

    new_pi = np.zeros( [env.observation_space.n, env.action_space.n] )

    maxQ = np.max(Q, axis=1)
    #print(maxQ)

    for s in range(env.observation_space.n) :
        q_cnt = 0

        for a in range(env.action_space.n) :
            if Q[s, a] == maxQ[s] :  # 至少有1
                q_cnt += 1

        for a in range(env.action_space.n) :
            if Q[s, a] == maxQ[s] :
                new_pi[s, a] = 1/q_cnt
            else :
                new_pi[s, a] = 0

    return new_pi

In [ ]:
def policy_improvement_epsilon_greedy(env, Q, epsilon=0.10) :

    new_pi = np.zeros( [env.observation_space.n, env.action_space.n] )

    maxQ = np.max(Q, axis=1)
    #print(maxQ)

    for s in range(env.observation_space.n) :
        q_cnt = 0

        for a in range(env.action_space.n) :
            if Q[s, a] == maxQ[s] :  # 至少有1
                q_cnt += 1

        for a in range(env.action_space.n) :
            if Q[s, a] == maxQ[s] :
                new_pi[s, a] = epsilon/env.action_space.n + ( (1-epsilon)/q_cnt )
            else :
                new_pi[s, a] = epsilon/env.action_space.n

    return new_pi

In [ ]:
def policy_iteration_1(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = policy_evaluation_Q(env, old_pi)
        print(Q)
        # new_pi >= old_pi
        new_pi = policy_improvement_greedy(env, Q)
        print(new_pi)

        # 在evaluation准确（Pi～Q），和improvement正确（Q～Pi）的条件下
        # 会生成Pi*（下一回合，Pi不再发生变化），Pi*会对应到值*

        # 是否存在值接近*，但是还没到值*，就improvement到Pi*（必然存在）
        # 此时，再对Pi*做evaluation，才可以得到值*

        # Pi～Q～Pi模式，检测Pi误差，确保了Pi*时的Q*

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def mc_iteration_1(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = mc_evaluation_Q(env, old_pi)
        print(Q)

        new_pi = policy_improvement_greedy(env, Q)
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def policy_iteration_2(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = policy_evaluation_Q(env, old_pi)
        print(Q)

        new_pi = policy_improvement_epsilon_greedy(env, Q)
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def mc_iteration_2(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = mc_evaluation_Q(env, old_pi)
        print(Q)

        new_pi = policy_improvement_epsilon_greedy(env, Q)
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def policy_iteration_3(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = policy_evaluation_Q(env, old_pi)
        print(Q)

        print(max(epsilon/np.log(2*cnt), 4e-2))
        new_pi = policy_improvement_epsilon_greedy(env, Q, max(epsilon/np.log(2*cnt), 4e-2))
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def mc_iteration_3(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = mc_evaluation_Q(env, old_pi)
        print(Q)

        print(max(epsilon/np.log(2*cnt), 4e-2))
        new_pi = policy_improvement_epsilon_greedy(env, Q, max(epsilon/np.log(2*cnt), 4e-2))
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def policy_iteration_4(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = policy_evaluation_Q(env, old_pi)
        print(Q)

        print(max(epsilon/cnt, 4e-2))
        new_pi = policy_improvement_epsilon_greedy(env, Q, max(epsilon/cnt, 4e-2))
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def mc_iteration_4(env, pi, epsilon=0.10, gamma=0.98, theta=1e-4) :

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)

    cnt = 0
    while True :
        cnt += 1

        diff = 0

        Q      = mc_evaluation_Q(env, old_pi)
        print(Q)

        print(max(epsilon/cnt, 4e-2))
        new_pi = policy_improvement_epsilon_greedy(env, Q, max(epsilon/cnt, 4e-2))
        print(new_pi)

        diff = np.sum(np.abs(new_pi - old_pi))

        old_pi = new_pi

        if diff < theta :
            print(cnt)
            break

    return old_pi

In [ ]:
def value_iteration(env, pi, epsilon=0.10, gamma=0.98, theta=1e-6) :

    old_V  = np.zeros(env.observation_space.n)
    new_V  = np.zeros(env.observation_space.n)

    old_Q  = np.zeros( [env.observation_space.n, env.action_space.n] )
    new_Q  = np.zeros( [env.observation_space.n, env.action_space.n] )

    old_pi = copy.deepcopy(pi)
    new_pi = copy.deepcopy(pi)


    cnt = 0
    while True :
        diff = 0

        for s in range(env.observation_space.n) :
            new_V[s] = 0
            for a in range(env.action_space.n) :
                new_Q[s, a] = 0

                # print(env.P[s][a])
                for p, s_next, r, done in env.P[s][a] :  # if s_next is done
                    new_Q[s, a] += p*(r+gamma*old_V[s_next]*(1-done))

            new_V[s] = np.max(new_Q[s])

            diff += np.abs(new_V[s] - old_V[s])

            old_V[s] = new_V[s]

        cnt += 1

        if diff < theta :
            print(cnt)
            break


    for s in range(env.observation_space.n) :
        for a in range(env.action_space.n) :
            new_Q[s, a] = 0

            # print(env.P[s][a])
            for p, s_next, r, done in env.P[s][a] :  # if s_next is done
                new_Q[s, a] += p*(r+gamma*old_V[s_next]*(1-done))
            old_Q[s, a] = new_Q[s, a]


    maxQ = np.max(old_Q, axis=1)
    #print(maxQ)
    for s in range(env.observation_space.n) :
        q_cnt = 0
        for a in range(env.action_space.n) :
            if old_Q[s, a] == maxQ[s] :
                q_cnt += 1

        for a in range(env.action_space.n) :
            if old_Q[s, a] == maxQ[s] :
                new_pi[s, a] = 1/q_cnt
            else :
                new_pi[s, a] = 0
            old_pi[s, a] = new_pi[s, a]


    print(old_V)
    print(old_Q)
    print(old_pi)

    return old_pi

In [ ]:
def policy_test(env, pi, ite_num=10000, ite_len=100) :

    suc_cnt = 0
    for i in range(ite_num) :

        obs, _ = env.reset()

        for j in range(ite_len) :

            action = np.random.choice(env.action_space.n, p=pi[obs])

            obs, reward, terminated, truncated, _ = env.step(action)

            if terminated or truncated :
                if reward == 1 :
                    suc_cnt += 1
                break

    print(suc_cnt/ite_num)

    return suc_cnt/ite_num

In [ ]:
env=gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
# gym.make('FrozenLake-v1', desc=generate_random_map(size=8))

# obs, info = env.reset(seed=42)
obs, info = env.reset()
# print(obs)
# print(info)

pi = equiprobability_policy(env)
# print(pi)

In [ ]:
print(policy_evaluation_V(env, pi))
print(mc_evaluation_V(env, pi))
print(TD0_evaluation_V(env, pi))
print(TDlambda_evaluation_V(env, pi))

In [ ]:
error21_avg = []
error31_avg = []
error41_avg = []


episodes_ary = np.arange(1000, 500001, 1000)
for epis in episodes_ary :

    V1 = policy_evaluation_V(env, pi)

    V2 = mc_evaluation_V(env, pi, epis)

    V3 = TD0_evaluation_V(env, pi, epis)

    V4 = TDlambda_evaluation_V(env, pi, epis)

    error21 = np.array(V2 - V1)
    error21_avg.append(np.mean(np.abs(error21)))

    error31 = np.array(V3 - V1)
    error31_avg.append(np.mean(np.abs(error31)))

    error41 = np.array(V4 - V1)
    error41_avg.append(np.mean(np.abs(error41)))


fig = plt.figure()
plt.plot(episodes_ary, error21_avg, label='mc-bs')
plt.plot(episodes_ary, error31_avg, label='td0-bs')
plt.plot(episodes_ary, error41_avg, label='tdn-bs')
plt.xlabel('episodes')
plt.ylabel('average error')
plt.legend()
plt.show()

In [ ]:
print(policy_evaluation_Q(env, pi))
print(mc_evaluation_Q(env, pi))
policy_test(env, pi)

In [ ]:
policy_test(env, policy_iteration_1(env, pi))
policy_test(env, mc_iteration_1(env, pi))
policy_test(env, policy_iteration_2(env, pi))
policy_test(env, mc_iteration_2(env, pi))
policy_test(env, policy_iteration_3(env, pi))
policy_test(env, mc_iteration_3(env, pi))
policy_test(env, policy_iteration_4(env, pi))
policy_test(env, mc_iteration_4(env, pi))

policy_test(env, value_iteration(env, pi))

In [ ]:
env.close()